# 📊 Visualizador Dinámico de Chunks - Celsia

Notebook para explorar y analizar los chunks procesados de Celsia de forma interactiva.

In [4]:
import json
import pandas as pd
import numpy as np
from IPython.display import display, HTML
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Configuración de visualización
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 100)
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

In [5]:
# 📂 Cargar datos de chunks
def load_chunks_data(filename='celsia_processed_20251015_223656_chunks.json'):
    """Carga los datos de chunks desde el archivo JSON"""
    try:
        with open(filename, 'r', encoding='utf-8') as f:
            data = json.load(f)
        print(f"✅ Datos cargados: {len(data)} chunks")
        return data
    except FileNotFoundError:
        print(f"❌ Archivo no encontrado: {filename}")
        return None
    except json.JSONDecodeError:
        print(f"❌ Error al decodificar JSON: {filename}")
        return None

# Cargar datos
chunks_data = load_chunks_data()
if chunks_data:
    df = pd.DataFrame(chunks_data)
    print(f"📊 DataFrame creado con {len(df)} filas y {len(df.columns)} columnas")
    print(f"📋 Columnas disponibles: {list(df.columns)}")

✅ Datos cargados: 2 chunks


ValueError: Mixing dicts with non-Series may lead to ambiguous ordering.

In [ ]:
# 🔍 Vista general de los datos
if chunks_data:
    print("📈 RESUMEN GENERAL")
    print("=" * 50)
    print(f"Total de chunks: {len(df)}")
    
    # Información por columnas
    for col in df.columns:
        non_null = df[col].notna().sum()
        print(f"{col}: {non_null}/{len(df)} valores no nulos")
    
    print("\n📊 PRIMERAS 5 FILAS:")
    display(df.head())

In [ ]:
# 🎯 Explorador Interactivo de Chunks
def show_chunk_details(index):
    """Muestra los detalles de un chunk específico"""
    if index < 0 or index >= len(df):
        print(f"❌ Índice fuera de rango. Use 0-{len(df)-1}")
        return
    
    chunk = df.iloc[index]
    
    print(f"🔍 CHUNK #{index}")
    print("=" * 80)
    
    for col in df.columns:
        value = chunk[col]
        if pd.notna(value):
            if col == 'content' and len(str(value)) > 200:
                print(f"📄 {col}: {str(value)[:200]}...")
            else:
                print(f"📋 {col}: {value}")
    print("=" * 80)

# Ejemplo: mostrar el primer chunk
if chunks_data and len(df) > 0:
    show_chunk_details(0)

In [ ]:
# 🔎 Búsqueda y filtrado de chunks
def search_chunks(keyword, column='content'):
    """Busca chunks que contengan una palabra clave"""
    if column not in df.columns:
        print(f"❌ Columna '{column}' no existe")
        return pd.DataFrame()
    
    mask = df[column].astype(str).str.contains(keyword, case=False, na=False)
    results = df[mask]
    
    print(f"🔍 Búsqueda: '{keyword}' en columna '{column}'")
    print(f"📊 Resultados encontrados: {len(results)}")
    
    return results

# Función para búsqueda interactiva
def interactive_search():
    keyword = input("🔍 Ingrese palabra clave para buscar: ")
    column = input(f"📋 Ingrese columna para buscar (default: content): ") or 'content'
    
    results = search_chunks(keyword, column)
    if len(results) > 0:
        print(f"\n📋 Primeros {min(5, len(results))} resultados:")
        display(results.head())
        
        if len(results) > 5:
            show_all = input(f"\n¿Mostrar todos los {len(results)} resultados? (y/n): ")
            if show_all.lower() == 'y':
                display(results)
    else:
        print("❌ No se encontraron resultados")

print("💡 Use interactive_search() para buscar chunks")
print("💡 Use search_chunks('palabra_clave', 'columna') para búsquedas directas")

In [ ]:
# 📈 Análisis estadístico de los chunks
if chunks_data:
    print("📊 ANÁLISIS ESTADÍSTICO")
    print("=" * 50)
    
    # Análisis de longitud de contenido
    if 'content' in df.columns:
        df['content_length'] = df['content'].astype(str).str.len()
        print(f"📏 Longitud de contenido:")
        print(f"  - Promedio: {df['content_length'].mean():.1f} caracteres")
        print(f"  - Mediana: {df['content_length'].median():.1f} caracteres")
        print(f"  - Mínimo: {df['content_length'].min()} caracteres")
        print(f"  - Máximo: {df['content_length'].max()} caracteres")
    
    # Análisis por categorías
    categorical_columns = df.select_dtypes(include=['object']).columns
    for col in categorical_columns:
        if col not in ['content'] and df[col].notna().sum() > 0:
            unique_values = df[col].value_counts()
            if len(unique_values) < 20:  # Solo mostrar si no hay demasiadas categorías
                print(f"\n📊 Distribución de {col}:")
                for value, count in unique_values.items():
                    print(f"  - {value}: {count} chunks")

In [ ]:
# 📊 Visualizaciones
if chunks_data and 'content_length' in df.columns:
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    fig.suptitle('📊 Análisis Visual de Chunks - Celsia', fontsize=16)
    
    # Histograma de longitud de contenido
    axes[0, 0].hist(df['content_length'], bins=30, alpha=0.7, color='skyblue')
    axes[0, 0].set_title('📏 Distribución de Longitud de Contenido')
    axes[0, 0].set_xlabel('Caracteres')
    axes[0, 0].set_ylabel('Frecuencia')
    
    # Box plot de longitud
    axes[0, 1].boxplot(df['content_length'])
    axes[0, 1].set_title('📦 Box Plot - Longitud de Contenido')
    axes[0, 1].set_ylabel('Caracteres')
    
    # Si hay columnas categóricas, hacer gráficos
    categorical_cols = [col for col in df.columns if df[col].dtype == 'object' and col != 'content']
    
    if len(categorical_cols) > 0:
        col = categorical_cols[0]
        if df[col].notna().sum() > 0:
            value_counts = df[col].value_counts().head(10)
            axes[1, 0].bar(range(len(value_counts)), value_counts.values)
            axes[1, 0].set_title(f'📊 Top 10 - {col}')
            axes[1, 0].set_xticks(range(len(value_counts)))
            axes[1, 0].set_xticklabels(value_counts.index, rotation=45, ha='right')
    
    # Timeline si hay fechas
    date_cols = [col for col in df.columns if 'date' in col.lower() or 'time' in col.lower()]
    if date_cols:
        col = date_cols[0]
        try:
            dates = pd.to_datetime(df[col], errors='coerce')
            dates = dates.dropna()
            if len(dates) > 0:
                axes[1, 1].hist(dates, bins=20, alpha=0.7, color='lightgreen')
                axes[1, 1].set_title(f'📅 Distribución Temporal - {col}')
                axes[1, 1].tick_params(axis='x', rotation=45)
        except:
            axes[1, 1].text(0.5, 0.5, 'No hay datos de fecha válidos', ha='center', va='center')
    else:
        axes[1, 1].text(0.5, 0.5, 'No hay columnas de fecha', ha='center', va='center')
    
    plt.tight_layout()
    plt.show()

In [ ]:
# 🎛️ Panel de Control Interactivo
def control_panel():
    """Panel de control para navegar por los chunks"""
    if not chunks_data:
        print("❌ No hay datos cargados")
        return
    
    while True:
        print("\n🎛️  PANEL DE CONTROL - CHUNKS CELSIA")
        print("=" * 50)
        print("1. 👁️  Ver chunk específico (por índice)")
        print("2. 🔍 Buscar chunks")
        print("3. 📊 Mostrar estadísticas")
        print("4. 📋 Listar primeros 10 chunks")
        print("5. 🎲 Ver chunk aleatorio")
        print("6. 💾 Exportar resultados filtrados")
        print("0. ❌ Salir")
        
        choice = input("\n➤ Seleccione una opción: ")
        
        if choice == '0':
            print("👋 ¡Hasta luego!")
            break
        elif choice == '1':
            try:
                index = int(input(f"Ingrese índice (0-{len(df)-1}): "))
                show_chunk_details(index)
            except ValueError:
                print("❌ Ingrese un número válido")
        elif choice == '2':
            interactive_search()
        elif choice == '3':
            display(df.describe())
        elif choice == '4':
            display(df.head(10))
        elif choice == '5':
            random_index = np.random.randint(0, len(df))
            print(f"🎲 Chunk aleatorio (índice {random_index}):")
            show_chunk_details(random_index)
        elif choice == '6':
            filename = input("Nombre del archivo (sin extensión): ") + ".csv"
            df.to_csv(filename, index=False, encoding='utf-8')
            print(f"✅ Datos exportados a: {filename}")
        else:
            print("❌ Opción no válida")

print("💡 Use control_panel() para acceder al panel de control interactivo")

In [ ]:
# 🚀 Análisis de calidad y duplicados
if chunks_data:
    print("🔍 ANÁLISIS DE CALIDAD Y DUPLICADOS")
    print("=" * 50)
    
    # Análisis de calidad si existe la columna
    if 'quality_score' in df.columns:
        quality_stats = df['quality_score'].describe()
        print("📊 Estadísticas de Calidad:")
        for stat, value in quality_stats.items():
            print(f"  - {stat}: {value:.3f}")
        
        # Chunks de alta y baja calidad
        high_quality = df[df['quality_score'] > 0.8]
        low_quality = df[df['quality_score'] < 0.5]
        print(f"\n🌟 Chunks de alta calidad (>0.8): {len(high_quality)}")
        print(f"⚠️  Chunks de baja calidad (<0.5): {len(low_quality)}")
    
    # Análisis de duplicados
    if 'content' in df.columns:
        duplicates = df['content'].duplicated().sum()
        print(f"\n🔄 Duplicados exactos encontrados: {duplicates}")
        
        # Chunks muy similares (por longitud)
        if 'content_length' in df.columns:
            similar_lengths = df.groupby('content_length').size()
            similar_lengths = similar_lengths[similar_lengths > 1]
            print(f"📏 Grupos con longitud similar: {len(similar_lengths)}")

## 🎯 Instrucciones de Uso

### Funciones Principales:

1. **`control_panel()`** - Panel interactivo para navegar
2. **`show_chunk_details(index)`** - Ver detalles de un chunk
3. **`search_chunks('keyword', 'column')`** - Buscar en los chunks
4. **`interactive_search()`** - Búsqueda interactiva

### Ejemplos de Uso:

```python
# Ver el chunk número 5
show_chunk_details(5)

# Buscar chunks que contengan "energía"
results = search_chunks('energía')

# Panel de control interactivo
control_panel()
```

### Tips:
- Los datos se cargan automáticamente al ejecutar las celdas
- Use el panel de control para una navegación fácil
- Las visualizaciones se generan automáticamente
- Puede exportar resultados filtrados a CSV